# 뉴욕타임즈

In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib.request

import pandas as pd
import datetime
import re
import time
import pickle

import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

nltk.download('punkt')
nltk.download('stopwords')

from pynytimes import NYTAPI

pd.set_option('display.max_colwidth', -1) # each column width maximum

In [13]:
NYT = NYTAPI("c0ZdIiQktA80J9QURItJP4Qp0IrosAIT", https=False)

In [14]:
ARTICLES = []

def get_nyt_api(period):
    for i in range(0, len(period)): # because of time difference between Korean and U.S server, start from 2nd value (not 1st value)
        ARTICLE = NYT.article_search(  
            query = KEYWORD,
            results = 50, # the number is meanless, but consider how many articles are issued in just one day
            dates = {
                "begin": period[i],
                "end": period[i]
            },
            options = {
                "sort": "oldest",
                "sources": [
                    "New York Times"
                ]
            }
        )
        ARTICLES.extend(ARTICLE)

In [15]:
START = datetime.datetime.strptime("2021, 03, 21", "%Y, %m, %d") # start date query
END = datetime.datetime.strptime("2021, 03, 22", "%Y, %m, %d") # end date query
DATE_VECTOR = [START + datetime.timedelta(days=x) for x in range(0, (END-START).days+1)] # dates as numeric vector

KEYWORD = 'human rights'

get_nyt_api(DATE_VECTOR)

In [16]:
len(ARTICLES)

8

In [17]:
ARTICLES[0]

{'abstract': 'In a new photo book, Chris Buck shares portraits and stories of strippers and their partners.',
 'web_url': 'https://www.nytimes.com/2021/03/21/style/gentlemens-club-chris-buck-strippers.html',
 'snippet': 'In a new photo book, Chris Buck shares portraits and stories of strippers and their partners.',
 'lead_paragraph': 'For his new photo book, “Gentlemen’s Club,” Chris Buck spent six years interviewing and taking portraits of strippers and their romantic partners.',
 'print_section': 'ST',
 'print_page': '4',
 'source': 'The New York Times',
 'multimedia': [{'rank': 0,
   'subtype': 'xlarge',
   'caption': None,
   'credit': None,
   'type': 'image',
   'url': 'images/2021/03/21/fashion/14STRIPPER-PARTNER-BOOK-05/14STRIPPER-PARTNER-BOOK-05-articleLarge.jpg',
   'height': 450,
   'width': 600,
   'legacy': {'xlarge': 'images/2021/03/21/fashion/14STRIPPER-PARTNER-BOOK-05/14STRIPPER-PARTNER-BOOK-05-articleLarge.jpg',
    'xlargewidth': 600,
    'xlargeheight': 450},
   'sub

In [24]:
with open('NYTAPI_TEST.pickle', 'wb') as f:
    pickle.dump(ARTICLES, f)

In [25]:
TITLE_OF_ARTICLE = []
URL_OF_ARTICLE = []
DATE_OF_ARTICLE = []

def get_nytimes_info(URLS):
    for i in range(0, len(URLS)):
        TITLE_OF_ARTICLE.append(URLS[i]['headline']['main']) # headline
        URL_OF_ARTICLE.append(URLS[i]['web_url']) # article link
        DATE_OF_ARTICLE.append(str(URLS[i]['pub_date'][0:10])) # publish time

get_nytimes_info(ARTICLES)

In [26]:
GET_LINK = pd.DataFrame(list(zip(TITLE_OF_ARTICLE, URL_OF_ARTICLE, DATE_OF_ARTICLE)), columns =['Title', 'Url', 'Date'])
GET_LINK.head()

,Title,Url,Date
0,The Private Lives of Strippers,https://www.nytimes.com/2021/03/21/style/gentlemens-club-chris-buck-strippers.html,2021-03-21
1,Excellence Runs in the Family. Her Novel’s Heroine Wants Something Else.,https://www.nytimes.com/2021/03/21/books/kaitlyn-greenidge-libertie.html,2021-03-21
2,Rich Countries Signed Away a Chance to Vaccinate the World,https://www.nytimes.com/2021/03/21/world/vaccine-patents-us-eu.html,2021-03-21
3,Second Canadian Is Tried in China on Spying Charges,https://www.nytimes.com/2021/03/22/world/asia/china-michael-kovrig-spying-canada.html,2021-03-22
4,Menthol Cigarettes Kill Many Black People. A Ban May Finally Be Near.,https://www.nytimes.com/2021/03/22/health/methol-smoking-ban.html,2021-03-22


In [31]:
GET_LINK.to_csv(f'{str(START)[0:10]}_{str(END)[0:10]}.csv', encoding = 'utf-8-sig', index_label = False, index = False)

In [32]:
URL_TAG = urllib.request.urlopen(ARTICLES[0]['web_url'])
SOURCE = BeautifulSoup(URL_TAG, 'lxml', from_encoding='utf-8')
URL_CONTENT = str(SOURCE.select('div.css-53u6y8 > p'))
URL_CONTENT = re.sub('<.+?>', '', URL_CONTENT, 0)
print(URL_CONTENT)

[For his new photo book, “Gentlemen’s Club,” Chris Buck spent six years interviewing and taking portraits of strippers and their romantic partners., “Partners of dancers are in this inherently complicated space of dating someone who’s intimate with others for their work,” he said. In that sense, the project is a study not only of labor, but also of relationships and fidelity., One question drove many of the interviews: How do you feel about your partner’s work?, “You shouldn’t be doing that. Wash dishes. Be a nurse. You’re a good girl. Take care of people,” Vincent from Jersey City, who is featured on the book’s cover, says in the book. (Mr. Buck included only first names.) Vincent adds that he doesn’t like his wife’s work, but it “pays the rent.”, Aaron, who is the main caregiver for two daughters, tells Mr. Buck he has struggled with his wife’s career. However, he says, “I get to spend all of my time with my girls, and it’s funded by her dancing.”, Others fully embrace the work. “I k

In [33]:
from urllib.error import HTTPError

TITLE_OF_ARTICLES = []
URL_OF_ARTICLES = []
DATE_OF_ARTICLES = []
CONTENT_OF_ARTICLES = []

def making_nyt_df(URLS):
    for i in range(0, len(ARTICLES)):
        TITLE_OF_ARTICLES.append(ARTICLES[i]['headline']['main']) 
        URL_OF_ARTICLES.append(ARTICLES[i]['web_url'])
        DATE_OF_ARTICLES.append(str(ARTICLES[i]['pub_date'][0:10]))

        REQ = urllib.request.Request(ARTICLES[i]['web_url'],
                                    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})
        try:
            SOURCE_CODE = urllib.request.urlopen(REQ)
            HTML_TAG = BeautifulSoup(SOURCE_CODE, 'lxml', from_encoding='utf-8')
            STRING_ITEM = ''
            for ITEM in HTML_TAG.select('div.css-53u6y8 > p'):
                STRING_ITEM += ITEM.get_text() # paragraph 단위로 있는 기사 본문을 stack처럼 쌓고 append 로 list 처리
            CONTENT_OF_ARTICLES.append(STRING_ITEM)
        except HTTPError as ERR: # 기사 링크는 있으나 기사 본문이 지워진 링크 처리
            if ERR.code == 404:
                CONTENT_OF_ARTICLES.append('404 NOT FOUND')

        if i % 20 == 0:
            time.sleep(2)
            print(f'{i}nd article complete')            

In [34]:
making_nyt_df(ARTICLES)

0nd article complete


In [36]:
NYTDATA = pd.DataFrame(list(zip(TITLE_OF_ARTICLES, URL_OF_ARTICLES, CONTENT_OF_ARTICLES, DATE_OF_ARTICLES)), columns =['Title', 'Url', 'Content', 'Date'])
NYTDATA

Title  \
0  The Private Lives of Strippers                                               
1  Excellence Runs in the Family. Her Novel’s Heroine Wants Something Else.     
2  Rich Countries Signed Away a Chance to Vaccinate the World                   
3  Second Canadian Is Tried in China on Spying Charges                          
4  Menthol Cigarettes Kill Many Black People. A Ban May Finally Be Near.        
5  Anti-Asian Attacks Place Andrew Yang in the Spotlight. How Will He Use It?   
6  U.S. Joins Allies to Punish Chinese Officials for Human Rights Abuses        
7  Your Tuesday Briefing                                                        

                                                                                            Url  \
0  https://www.nytimes.com/2021/03/21/style/gentlemens-club-chris-buck-strippers.html             
1  https://www.nytimes.com/2021/03/21/books/kaitlyn-greenidge-libertie.html                       
2  https://www.nytimes.com/2021/03/21/world/vaccine-patents-us-eu.html                            
3  https://www.nytimes.com/2021/03/22/world/asia/china-michael-kovrig-spying-canada.html          
4  https://www.nytimes.com/2021/03/22/health/methol-smoking-ban.html                              
5  https://www.nytimes.com/2021/03/22/nyregion/andrew-yang-bias-asian-atlanta.html                
6  https://www.nytimes.com/2021/03/22/us/politics/sanctions-china-uighurs.html                    
7  https://www.nytimes.com/2021/03/22/briefing/vaccine-inequity-xinjiang-sanctions-uyghurs.html   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [37]:
del_raw = NYTDATA[NYTDATA['Content'] == ''].index # 기사본문이 들어간 Content 열에서 내용이 없이 '' 로 확인되는 열 인덱싱
NYTDATA_PRE = NYTDATA.drop(del_raw)

print(NYTDATA.shape[0]) # 변경된 데이터프레임 행 개수 확인
print(NYTDATA_PRE.shape[0])

8
8


In [40]:
NYTDATA_PRE.to_csv(f'{str(START)[0:10]}_{str(END)[0:10]}.csv', encoding = 'utf-8-sig',
                index_label = False, index = False)

# 워싱턴포스트

In [43]:
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage') 
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36")

In [44]:
from bs4 import BeautifulSoup
import requests
import urllib.request
from urllib.request import HTTPError
import pandas as pd
import re
import time

import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

nltk.download('punkt')
nltk.download('stopwords')

pd.set_option('display.max_colwidth', -1)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LEE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LEE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-44-7e5653353902>:21: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [51]:
KEYWORD = 'BTS'

LINK = f'https://www.washingtonpost.com/newssearch/?query={KEYWORD}&btn-search=&sort=Relevance&datefilter=All%20Since%202005&contenttype=Article&spellcheck&startat='
LINK

'https://www.washingtonpost.com/newssearch/?query=BTS&btn-search=&sort=Relevance&datefilter=All%20Since%202005&contenttype=Article&spellcheck&startat='

In [55]:
TITLES = []
TITLES_LINKS = []
DATES = []

def get_link_from_news_title(url, article_num):
    NUM = int(article_num/20 + 1) 
    for i in range(0, NUM):
        n = 20*i
        URL = LINK + str(n)
        DRIVER = webdriver.Chrome(executable_path=r'C:/chromedriver/chromedriver.exe')
        DRIVER.implicitly_wait(3) #페이지가 뜨는데 걸리는 시간 3초
        DRIVER.get(URL)

        SEARCH_RESULTS = DRIVER.find_elements_by_css_selector('div.pb-feed-headline.ng-scope')
        for RESULT in SEARCH_RESULTS:
            TITLES.append(RESULT.find_element_by_tag_name('a').text)
            TITLES_LINKS.append(RESULT.find_element_by_tag_name('a').get_attribute('href'))
                
        ARTICLE_DATE = DRIVER.find_elements_by_css_selector('div.pb-feed-byline.ng-binding')
        for DATE in ARTICLE_DATE:
            DATES.append(DATE.find_element_by_tag_name('span').text)

    if n % 40 == 0:
        time.sleep(2)
        print(f'{n}nd webpage completed')


In [57]:
get_link_from_news_title(LINK, 166) # 워싱턴 포스트에서 검색 키워드로 전체 기사가 몇 개가 나오는지를 확인한 뒤(위에서 LINK 변수) 숫자 입력

print(len(TITLES))
print(len(TITLES_LINKS))
print(len(DATES))

160nd webpage completed
236
236
236


In [58]:
WP = pd.DataFrame(list(zip(TITLES, TITLES_LINKS, DATES)), columns =['Title', 'Url', 'Date'])
WP.head()

,Title,Url,Date
0,What to expect from the 2021 Grammy Awards,https://www.washingtonpost.com/arts-entertainment/2021/03/10/2021-grammy-awards-performers-nominees-date/,"Mar 12, 2021"
1,The Grammys are the latest award show to see a drastic drop in TV ratings,https://www.washingtonpost.com/arts-entertainment/2021/03/17/grammys-golden-globe-ratings-down/,"Mar 17, 2021"
2,"For fans of BTS, the only thing better than the group is each other",https://www.washingtonpost.com/arts-entertainment/2020/11/20/bts-be-kpop-army-fans/,"Nov 20, 2020"
3,PHOTOS: See music’s return to the big stage at the Grammys,https://www.washingtonpost.com/health/photos-see-musics-return-to-the-big-stage-at-the-grammys/2021/03/15/848e9ff6-8557-11eb-be4a-24b89f616f2c_story.html,"Mar 15, 2021"
4,"Beyoncé, on track to make history, wins 2 early Grammys",https://www.washingtonpost.com/national/beyonce-taylor-swift-could-have-historic-night-at-grammys/2021/03/14/a1b8c36c-848b-11eb-be22-32d331d87530_story.html,"Mar 15, 2021"


In [59]:
WP.to_csv('WP_links.csv', encoding = 'utf-8-sig',
                index_label = False, index = False)

In [60]:
TITLES = WP['Title']
TITLES_LINKS = WP['Url']
DATES = WP['Date']

In [64]:
CONTENTS = []

def get_text(url):
    for i in range(0, len(url)):
        try:
            REQ = urllib.request.Request(url[i], headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'})
            SOURCE = urllib.request.urlopen(REQ)
            SOUP = BeautifulSoup(SOURCE, features='lxml', from_encoding='utf-8')
            ITEM = str(SOUP.select('div.article-body')) # contents
            URL_CONTENT = re.sub('<.+?>', '', ITEM)
            URL_CONTENT = re.sub('\n', '', URL_CONTENT)
            CONTENTS.append(URL_CONTENT)

            # 2005~2010년 기사들의 경우 html 구조가 최신 기사들과 달라 별도의 방식으로 처리함
            # 해당 기사들은 일종의 header와 body 형태로 구분되어 있어 두 개를 따로 크롤링한 뒤 합치는 과정을 거친다

            if URL_CONTENT == '[]': #아무 것도 없으면 문자열을 만들어 무언가 넣을 준비를 해라.....
                ITEM = str(SOUP.select('div#article_body > p')) # 헤더 부분 id는 #으로 표시...
                URL_CONTENT = re.sub('<.+?>', '', ITEM) # +는 조합을 의미한다....p, div 상관없이 소스코드 모두 없애주는...
                URL_CONTENT = re.sub('\n', '', URL_CONTENT) # 문단은 엔터로 구별되어 있으니, 없애라....

                ITEM1 = str(SOUP.select('div#body_after_content_column > p')) # 바디 부분 id는 #으로 표시...
                URL_CONTENT1 = re.sub('<.+?>', '', ITEM1) # +는 조합을 의미한다....p, div 상관없이 소스코드 모두 없애주는...
                URL_CONTENT1 = re.sub('\n', '', URL_CONTENT1) # 문단은 엔터로 구별되어 있으니, 없애라....

                CONTENTS[i] = URL_CONTENT + " " + URL_CONTENT1
        
        except URLError: # 일부 기사들의 경우 지나치게 자주 조회가 되면 서버에서 조회를 막아버리기 때문에 이를 패싱하는 코드를 입력한다
            CONTENTS.append('110 Server not connected')

        if i % 40 == 0:
            time.sleep(2) # 기사 40개당 2초 쉬기 / 기사 수집 중에 ip가 차단될 경우 횟수를 줄여서 sleep timing의 간격을 짧게 할 것
            print(f'{i}nd item is collected')
    
    print(len(CONTENTS))

# 팁

* id는 #으로
* class는 .으로


In [65]:
get_text(TITLES_LINKS)

0nd item is collected
40nd item is collected
80nd item is collected
120nd item is collected
160nd item is collected
200nd item is collected
236


In [66]:
print(len(CONTENTS))

WP_DATA = pd.DataFrame(list(zip(TITLES, TITLES_LINKS, DATES, CONTENTS)), columns =['Title', 'Url', 'Date', 'Text'])
WP_DATA.head()

236


Title  \
0  What to expect from the 2021 Grammy Awards                                  
1  The Grammys are the latest award show to see a drastic drop in TV ratings   
2  For fans of BTS, the only thing better than the group is each other         
3  PHOTOS: See music’s return to the big stage at the Grammys                  
4  Beyoncé, on track to make history, wins 2 early Grammys                     

                                                                                                                                                            Url  \
0  https://www.washingtonpost.com/arts-entertainment/2021/03/10/2021-grammy-awards-performers-nominees-date/                                                      
1  https://www.washingtonpost.com/arts-entertainment/2021/03/17/grammys-golden-globe-ratings-down/                                                                
2  https://www.washingtonpost.com/arts-entertainment/2020/11/20/bts-be-kpop-army-fans/                                                                            
3  https://www.washingtonpost.com/health/photos-see-musics-return-to-the-big-stage-at-the-grammys/2021/03/15/848e9ff6-8557-11eb-be4a-24b89f616f2c_story.html      
4  https://www.washingtonpost.com/national/beyonce-taylor-swift-could-have-historic-night-at-grammys/2021/03/14/a1b8c36c-848b-11eb-be22-32d331d87530_story.html   

           Date  \
0  Mar 12, 2021   
1  Mar 17, 2021   
2  Nov 20, 2020   
3  Mar 15, 2021   
4  Mar 15, 2021   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [67]:
WP_DATA.to_csv('WP_DB.csv', encoding = 'utf-8-sig',                index_label = False, index = False)